In [1]:
import torch
from policy_networks.mlp import PolicyNet
from policy_networks.rnn import RNNPolicyNet
from torch import optim
from tasks import RandomMaze
import numpy as np
from tasks import Maze_config
from gvf_networks.tabular import Tabular
from utils import train_epoch
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# get gpu when available
if torch.cuda.is_available():
    use_gpu = True
    device = torch.device("cuda:0")
    print("GPU is available", flush=True)
else:
    use_gpu = False
    device = torch.device("cpu")
    print("no GPU found", flush=True)

no GPU found


In [3]:
# parameters
gammas = [0.98]
num_actions = 4
lr_adam = 5e-4
env_name = 'RandomMaze-v0'
size = 20
depth = 5
lr = {'sr': 0.3, 'gvfs': 0.6, 'Q': 0.1}
thresholds = {'sr': 0.6, 'gvfs': 0.1}
change = 50  # maze is changed every change epochs
batch_size = 1000
nb_epochs = 10000
nb_episodes_random_policy = 20

# for plotting purposes
x_grid,y_grid = np.meshgrid(np.linspace(20,480,size),np.linspace(20,480,size))

In [4]:
# declare gvf network
gvf_net = Tabular(size=size, num_actions=num_actions, depth=depth, gammas=gammas, 
                  thresholds=thresholds, lr=lr)

# declare policy network
logits_net = RNNPolicyNet(4 + (depth + 1) * num_actions * len(gammas), num_actions).to(device)

# make optimizer
optimizer = optim.Adam(logits_net.parameters(), lr=lr_adam)

# define environment
env, L = RandomMaze.main(start_position='goal_position',env_name=env_name, size=size)

In [5]:
total_reward = []

# training loop
for i in range(nb_epochs):
    clear_output(wait=True)
    
    # new network
    if i==0 or i%change==0:
        # reset gvf network
        gvf_net.reset()
        # new environment
        env, L = RandomMaze.main(start_position='goal_position',env_name=env_name, size=size)    
    elif np.any([(i-k)%change == 0 for k in range(1, 3)]): # for first 3 episodes, start at goal
        # set start position to goal
        Maze_config.start_idx = Maze_config.goal_idx
    else:
        # random new position
        Maze_config.start_idx = [L[np.random.randint(0, len(L))]]
        
    # training
    batch_loss, batch_rets, batch_lens = train_epoch(i, change, batch_size, env, gvf_net, logits_net, 
                                                     optimizer, device, nb_episodes_random_policy)

    print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
            (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))

    if np.all([(i - k) % change != 0 for k in range(nb_episodes_random_policy)]):
        total_reward.append(np.mean(batch_lens))

    policy_all_states = logits_net.get_policy_all_states(env, gvf_net, device)

    # figure of gvfs        
    img = env.render('rgb_array')     
    for i_plot in range(20):
        plt.subplot(5, 4, i_plot + 1)
        plt.imshow(gvf_net.gvfs[int(i_plot/4), :, :, -1, int(i_plot%4)])
        if i_plot <= 3:
            plt.title(np.array(['down', 'up', 'left', 'right'])[i_plot])
        plt.colorbar()

    # figure of policy network
    plt.figure()
    plt.imshow(img)
    plt.quiver(x_grid, 
               y_grid, 
               policy_all_states[:,:,3] - policy_all_states[:,:,2],
               policy_all_states[:,:,0] - policy_all_states[:,:,1])

    # figure of reward
    plt.figure()
    plt.plot(total_reward)
    plt.show()


TypeError: tuple indices must be integers or slices, not tuple

In [ ]:
self = logits_net

In [ ]:
size = gvf_net.size
policy_all_states = np.zeros([size, size, gvf_net.num_actions])
for x_pos in range(size):
    for y_pos in range(size):
        pos = np.array([x_pos, y_pos])
        r_pos = np.squeeze(env.maze.objects.goal.positions).copy()
        obs = torch.as_tensor(format_input(pos, r_pos, size, gvf_net), dtype=torch.float).to(device)
        policy_all_states[x_pos, y_pos] = self.get_policy(obs, h).probs.cpu().detach().numpy()

In [ ]:
obs

In [ ]:
import numpy as np
import torch


def reward_to_go(rewards):
    n = len(rewards)
    rtgs = np.zeros_like(rewards)
    for i in reversed(range(n)):
        rtgs[i] = rewards[i] + (rtgs[i + 1] if i + 1 < n else 0)
    return rtgs


def format_input(pos, r_pos, size, gvf_net) -> np.array:
    return np.array([(pos[0] - size / 2) / 25, (pos[1] - size / 2) / 25,
           (r_pos[0] - size / 2) / 25, (r_pos[1] - size / 2) / 25] + \
           gvf_net.gvfs[:, pos[0], pos[1], :, :].flatten().tolist() + \
           gvf_net.Q_gamma[pos[0], pos[1], :, :].flatten().tolist())
